In [1]:
import time
import os
import random
import numpy as np
import networkx as nx
import torch.nn.functional as F
import matplotlib.pyplot as plt

import torchvision
# Torch
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler

# Torchvison
from torchvision.transforms import transforms
import torchvision.models as models
from torchvision.datasets import CIFAR100, CIFAR10, Caltech101,Cityscapes

# Utils
import visdom
from tqdm import tqdm

# Custom
import models.resnet as resnet
import models.lossnet as lossnet
from config import *
from data.sampler import SubsetSequentialSampler
import copy
import pickle

import arguments
from utils import *
from sklearn.metrics import accuracy_score
from sklearn import metrics
from scipy import sparse
from dgl import DGLGraph
from dgl.data import register_data_args, load_data
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from gcn import GCN
from vat import VATLoss
from imutils import paths
import cv2
import pretrainedmodels


Using backend: pytorch
/usr/local/lib/python3.6/dist-packages/dgl/base.py:25: UserWarning: Detected an old version of PyTorch. Suggest using torch>=1.2.0 for the best experience.
  warnings.warn(msg, warn_type)


In [2]:
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    # torch.backends.cudnn.deterministic = True
    # os.environ['PYTHONHASHSEED']=str(SEED)
    # torch.backends.cudnn.benchmark = False # as all the inputs are not of same size
SEED=42
seed_everything(SEED=SEED)

RGB_MEAN = [0.5429, 0.5263, 0.4994]
RGB_STD = [0.2422, 0.2392, 0.2406]

In [3]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

epochs = 100

In [4]:
image_paths = list(paths.list_images('./caltech/101_ObjectCategories'))

data = []
labels = []
label_names = []
for image_path in image_paths:
    label = image_path.split(os.path.sep)[-2]
    if label == 'BACKGROUND_Google':
        continue

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    data.append(image)
    label_names.append(label)
    labels.append(label)
data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(len(lb.classes_))

(X, x_val , Y, y_val) = train_test_split(data, labels, 
                                                    test_size=0.2,  
                                                    stratify=labels,
                                                    random_state=42)

(x_train, x_test, y_train, y_test) = train_test_split(X, Y, 
                                                    test_size=0.25, 
                                                    random_state=42)

print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}\nx_val examples: {x_val.shape}")

# define transforms
# train_transform = transforms.Compose(
#     [transforms.ToPILImage(),
# 	 transforms.Resize((224, 224)),
#     #  transforms.RandomRotation((-30, 30)),
#     #  transforms.RandomHorizontalFlip(p=0.5),
#     #  transforms.RandomVerticalFlip(p=0.5),
#      transforms.ToTensor(),
#      transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                           std=[0.229, 0.224, 0.225])])

train_transform = transforms.Compose([
                            transforms.RandomResizedCrop(256, (.8, 1)),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomRotation(degrees=15),
                            transforms.ColorJitter(),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            transforms.Normalize(RGB_MEAN, RGB_STD),
                            ])
val_transform = transforms.Compose(
    [transforms.ToPILImage(),
	 transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])

class ImageDataset(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i][:]
        
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data
 
train_data = ImageDataset(x_train, y_train, train_transform)
val_data = ImageDataset(x_val, y_val, val_transform)
test_data = ImageDataset(x_test, y_test, val_transform)
 
# dataloaders
trainloader = DataLoader(train_data, batch_size=16, shuffle=True,num_workers=16)
valloader = DataLoader(val_data, batch_size=16, shuffle=True,num_workers=16)
testloader = DataLoader(test_data, batch_size=16, shuffle=False,num_workers=16)



101
x_train examples: (5205,)
x_test examples: (1736,)
x_val examples: (1736,)


In [5]:
train_transform = transforms.Compose(
    [transforms.ToPILImage(),
	 transforms.Resize((224, 224)),
#      transforms.RandomCrop(224),
#      transforms.RandomHorizontalFlip(),
    #  transforms.RandomRotation((-30, 30)),
    #  transforms.RandomHorizontalFlip(p=0.5),
    #  transforms.RandomVerticalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])

# train_transform = transforms.Compose([
#                             transforms.ToPILImage(),
#                             transforms.RandomResizedCrop(256, (.8, 1)),
#                             transforms.RandomHorizontalFlip(),
#                             transforms.RandomRotation(degrees=15),
#                             transforms.ColorJitter(),
#                             transforms.CenterCrop(224),
#                             transforms.ToTensor(),
#                             transforms.Normalize(RGB_MEAN, RGB_STD),
#                             ])
val_transform = transforms.Compose(
    [transforms.ToPILImage(),
	 transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])

class ImageDataset(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i][:]
        
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data
 
train_data = ImageDataset(x_train, y_train, train_transform)
val_data = ImageDataset(x_val, y_val, val_transform)
test_data = ImageDataset(x_test, y_test, val_transform)
 
# dataloaders
trainloader = DataLoader(train_data, batch_size=128, shuffle=True,num_workers=16)
valloader = DataLoader(val_data, batch_size=128, shuffle=True,num_workers=16)
testloader = DataLoader(test_data, batch_size=128, shuffle=False,num_workers=16)

In [6]:
def train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss, vis=None, plot_data=None):
    models['backbone'].train()
    # models['module'].train()
    global iters

    for data in tqdm(dataloaders['train'], leave=False, total=len(dataloaders['train'])):
        inputs = data[0].cuda()
        labels = data[1].cuda()
        # print(inputs.shape, labels.shape)
#         iters += 1

        optimizers['backbone'].zero_grad()
        # optimizers['module'].zero_grad()

#         scores, features, for_gcn = models['backbone'](inputs)
        scores = models['backbone'](inputs)
#         print(scores.shape)
#         print(torch.max(labels, 1)[1])
        target_loss = criterion(scores, torch.max(labels, 1)[1])

        # if epoch > epoch_loss:
        #     # After 120 epochs, stop the gradient from the loss prediction module propagated to the target model.
        #     features[0] = features[0].detach()
        #     features[1] = features[1].detach()
        #     features[2] = features[2].detach()
        #     features[3] = features[3].detach()
        # pred_loss = models['module'](features)
        # pred_loss = pred_loss.view(pred_loss.size(0))

        # m_backbone_loss = torch.sum(target_loss) / target_loss.size(0)
        # m_module_loss   = LossPredLoss(pred_loss, target_loss, margin=MARGIN)
        # loss            = m_backbone_loss + WEIGHT * m_module_loss

        target_loss.backward()
        optimizers['backbone'].step()
        # optimizers['module'].step()

        # Visualize
        # if (iters % 100 == 0) and (vis != None) and (plot_data != None):
        #     plot_data['X'].append(iters)
        #     plot_data['Y'].append([
        #         m_backbone_loss.item(),
        #         m_module_loss.item(),
        #         loss.item()
        #     ])
        #     vis.line(
        #         X=np.stack([np.array(plot_data['X'])] * len(plot_data['legend']), 1),
        #         Y=np.array(plot_data['Y']),
        #         opts={
        #             'title': 'Loss over Time',
        #             'legend': plot_data['legend'],
        #             'xlabel': 'Iterations',
        #             'ylabel': 'Loss',
        #             'width': 1200,
        #             'height': 390,
        #         },
        #         win=1
        #     )

#
def test(models, dataloaders, mode='val'):
    assert mode == 'val' or mode == 'test'
    models['backbone'].eval()
    # models['module'].eval()

    total = 0
    correct = 0
    with torch.no_grad():
        for (inputs, labels) in dataloaders[mode]:
            inputs = inputs.cuda()
            labels = labels.cuda()
            labels = torch.max(labels, 1)[1]
#             scores, _, _ = models['backbone'](inputs)
            scores = models['backbone'](inputs)
            _, preds = torch.max(scores.data, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    
    return 100 * correct / total

#
def train(models, criterion, optimizers, schedulers, dataloaders, num_epochs, epoch_loss, vis, plot_data, cycle, accuracies):
    print('>> Train a Model.')
    best_acc = 0.
    checkpoint_dir = os.path.join('./cifar10', 'train', 'weights')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    for epoch in range(num_epochs):
        schedulers['backbone'].step()
        # schedulers['module'].step()

        train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss, vis, plot_data)
        print(accuracies, epoch)

        # Save a checkpoint
        acc = test(models, dataloaders, 'test')
        print(acc)
        # if best_acc < acc:
        #     best_acc = acc
        #     best_model = copy.deepcopy(models['backbone'])
        # # test_acc = test(models, dataloaders, 'test')
        # print(acc, 'val_acc', best_acc, 'best_acc', 'accs', accuracies, 'epoch', epoch, 'cycle' ,cycle)



        # if False and epoch % 5 == 4:
        #     acc = test(models, dataloaders, 'test')
        #     if best_acc < acc:
        #         best_acc = acc
        #         torch.save({
        #             'epoch': epoch + 1,
        #             'state_dict_backbone': models['backbone'].state_dict(),
        #             'state_dict_module': models['module'].state_dict()
        #         },
        #         '%s/active_resnet18_cifar10.pth' % (checkpoint_dir))
        #     print('Val Acc: {:.3f} \t Best Acc: {:.3f}'.format(acc, best_acc))

    torch.save(models['backbone'].state_dict(), "model_" + str(cycle) + ".pt")
    print('>> Finished.')
# args = arguments.get_args()
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"
accuracies = []
# main(args)
# vis = visdom.Visdom(server='http://localhost', port=9000)
vis = None
plot_data = {'X': [], 'Y': [], 'legend': ['Backbone Loss', 'Module Loss', 'Total Loss']}

In [13]:
class ResNet18(nn.Module):
    def __init__(self, pretrained):
        super(ResNet18, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__['resnet18'](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict__['resnet18'](pretrained=None)
        
        # change the classification layer
        self.l0 = nn.Linear(512, len(lb.classes_))
        self.dropout = nn.Dropout2d(0.5)
    def forward(self, x):
        # get the batch size only, ignore (c, h, w)
        batch, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        x = self.dropout(x)
        l0 = self.l0(x)
        return l0

In [ ]:
for trial in range(1):
    # Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.

#     all_indices = set(np.arange(NUM_TRAIN))
#     val_indices = random.sample(all_indices, 0)
#     all_indices = np.setdiff1d(list(all_indices), val_indices)

#     initial_indices = random.sample(list(all_indices), ADDENDUM)
#     f = open("./init_indice.pkl", 'wb')
#     pickle.dump(initial_indices, f)
    # indices = all_indices
    # random.shuffle(indices)
    # labeled_set = indices[:ADDENDUM]

    # unlabeled_set = indices[ADDENDUM:]

#     current_indices = list(initial_indices)

    train_loader = trainloader
    test_loader  = testloader
    val_loader = valloader
    dataloaders  = {'train': train_loader, 'test': test_loader, 'val': val_loader}

    # Model
    # resnet18    = resnet.ResNet18(num_classes=10).cuda()
    # # loss_module = lossnet.LossNet().cuda()
    # models      = {'backbone': resnet18}
    # torch.backends.cudnn.benchmark = False

    # Active learning cycles

    for cycle in range(1):
        # Loss, criterion and scheduler (re)initialization

        # Randomly sample 10000 unlabeled data points
        # random.shuffle(unlabeled_set)
#         unlabeled_indices = np.setdiff1d(list(all_indices), current_indices)
#         # Create unlabeled dataloader for the unlabeled subset
#         unlabeled_loader = DataLoader(train_data, batch_size=BATCH, 
#                                       sampler=SubsetSequentialSampler(unlabeled_indices), # more convenient if we maintain the order of subset
#                                       pin_memory=True)

        resnet18    = ResNet18(pretrained=True).cuda()
#         resnet18 = nn.DataParallel(resnet18)
#         resnet18.cuda()
        # loss_module = lossnet.LossNet().cuda()
        models      = {'backbone': resnet18}
        criterion      = nn.CrossEntropyLoss()
        optim_backbone = optim.SGD(models['backbone'].parameters(), lr=LR, 
                                momentum=MOMENTUM, weight_decay=WDECAY)
        # optim_module   = optim.SGD(models['module'].parameters(), lr=LR, 
        #                         momentum=MOMENTUM, weight_decay=WDECAY)
        sched_backbone = lr_scheduler.MultiStepLR(optim_backbone, milestones=MILESTONES)
        # sched_module   = lr_scheduler.MultiStepLR(optim_module, milestones=MILESTONES)

        optimizers = {'backbone': optim_backbone}
        schedulers = {'backbone': sched_backbone}

        # Training and test
        train(models, criterion, optimizers, schedulers, dataloaders, EPOCH, EPOCHL, vis, plot_data, 1, accuracies)
        acc = test(models, dataloaders, mode='test')
        accuracies.append(acc)
        print(acc)

  0%|          | 0/41 [00:00<?, ?it/s]

>> Train a Model.


[74.42396313364056, 73.963133640553] 0


  0%|          | 0/41 [00:00<?, ?it/s]

5.529953917050691


...

  0%|          | 0/41 [00:00<?, ?it/s]

68.66359447004608


[74.42396313364056, 73.963133640553] 95


  0%|          | 0/41 [00:00<?, ?it/s]

67.51152073732719


In [ ]:
print(MILESTONES,LR,BATCH)

In [2]:
class ResNet34(nn.Module):
    def __init__(self, pretrained):
        super(ResNet34, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained=None)
        
        self.l0 = nn.Linear(512, len(lb.classes_))
        self.dropout = nn.Dropout2d(0.4)

    def forward(self, x):
        # get the batch size only, ignore (c, h, w)
        batch, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        x = self.dropout(x)
        l0 = self.l0(x)
        return l0

model = ResNet34(pretrained=True).to(device)
print(model)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-2)
# loss function
criterion = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/admin/.cache/torch/checkpoints/resnet34-333f7ec4.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:852)>

In [30]:
print(f"Training on {len(train_data)} examples, validating on {len(val_data)} examples")

#validation function
def validate(model, dataloader):
    print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            outputs = model(data)
            loss = criterion(outputs, torch.max(target, 1)[1])
            
            val_running_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            val_running_correct += (preds == torch.max(target, 1)[1]).sum().item()
        
        val_loss = val_running_loss/len(dataloader.dataset)
        val_accuracy = 100. * val_running_correct/len(dataloader.dataset)
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')
        
        return val_loss, val_accuracy

# training function
def fit(model, dataloader):
    print('Training')
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(data)
        # print(outputs)
        # print(torch.max(target, 1)[1])
        loss = criterion(outputs, torch.max(target, 1)[1])
        train_running_loss += loss.item()
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == torch.max(target, 1)[1]).sum().item()
        loss.backward()
        optimizer.step()
        
    train_loss = train_running_loss/len(dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(dataloader.dataset)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}")
    
    return train_loss, train_accuracy


train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []
start = time.time()
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
    val_epoch_loss, val_epoch_accuracy = validate(model, valloader)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
end = time.time()

print((end-start)/60, 'minutes')

Training on 5205 examples, validating on 1736 examples
Epoch 1 of 100
Training



 99%|█████████▉| 322/325 [00:12<00:00, 26.53it/s]
326it [00:12, 25.61it/s]                         

Train Loss: 0.2782, Train Acc: 13.10
Validating




 90%|████████▉ | 97/108 [00:01<00:00, 68.04it/s]
109it [00:01, 56.82it/s]                         

Val Loss: 0.2687, Val Acc: 16.94
Epoch 2 of 100
Training


...



 90%|████████▉ | 97/108 [00:01<00:00, 67.15it/s]
109it [00:01, 57.45it/s]                         

Val Loss: 0.1832, Val Acc: 65.26
Epoch 93 of 100
Training




 99%|█████████▉| 323/325 [00:12<00:00, 26.52it/s]
326it [00:12, 25.23it/s]                         

Train Loss: 0.0023, Train Acc: 98.87
Validating




 86%|████████▌ | 93/108 [00:01<00:00, 67.40it/s]
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
from PIL import Image
image = Image.open(image_path)

In [6]:
img = image.convert('RGB')

In [17]:
def train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss, vis=None, plot_data=None):
    models['backbone'].train()
    # models['module'].train()
    global iters

    for data in tqdm(dataloaders['train'], leave=False, total=len(dataloaders['train'])):
        inputs = data[0].cuda()
        labels = data[1].cuda()
        # print(inputs.shape, labels.shape)
        iters += 1
#         print(inputs.shape)
        optimizers['backbone'].zero_grad()
        # optimizers['module'].zero_grad()
        models['backbone'].cuda()
        scores, features, for_gcn = models['backbone'](inputs)
        target_loss = criterion(scores, torch.max(labels, 1)[1].cuda())

        # if epoch > epoch_loss:
        #     # After 120 epochs, stop the gradient from the loss prediction module propagated to the target model.
        #     features[0] = features[0].detach()
        #     features[1] = features[1].detach()
        #     features[2] = features[2].detach()
        #     features[3] = features[3].detach()
        # pred_loss = models['module'](features)
        # pred_loss = pred_loss.view(pred_loss.size(0))

        # m_backbone_loss = torch.sum(target_loss) / target_loss.size(0)
        # m_module_loss   = LossPredLoss(pred_loss, target_loss, margin=MARGIN)
        # loss            = m_backbone_loss + WEIGHT * m_module_loss

        target_loss.backward()
        optimizers['backbone'].step()
        # optimizers['module'].step()

        # Visualize
        # if (iters % 100 == 0) and (vis != None) and (plot_data != None):
        #     plot_data['X'].append(iters)
        #     plot_data['Y'].append([
        #         m_backbone_loss.item(),
        #         m_module_loss.item(),
        #         loss.item()
        #     ])
        #     vis.line(
        #         X=np.stack([np.array(plot_data['X'])] * len(plot_data['legend']), 1),
        #         Y=np.array(plot_data['Y']),
        #         opts={
        #             'title': 'Loss over Time',
        #             'legend': plot_data['legend'],
        #             'xlabel': 'Iterations',
        #             'ylabel': 'Loss',
        #             'width': 1200,
        #             'height': 390,
        #         },
        #         win=1
        #     )

In [18]:
def train(models, criterion, optimizers, schedulers, dataloaders, num_epochs, epoch_loss, vis, plot_data, cycle, accuracies):
    print('>> Train a Model.')
    best_acc = 0.
    checkpoint_dir = os.path.join('./cifar10', 'train', 'weights')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
#     for epoch in range(num_epochs):
    schedulers['backbone'].step()
    # schedulers['module'].step()

    train_epoch(models, criterion, optimizers, dataloaders, 1, epoch_loss, vis, plot_data)
#     print(accuracies, epoch)

    # Save a checkpoint
    # acc = test(models, dataloaders, 'val')
    # if best_acc < acc:
    #     best_acc = acc
    #     best_model = copy.deepcopy(models['backbone'])
    # # test_acc = test(models, dataloaders, 'test')
    # print(acc, 'val_acc', best_acc, 'best_acc', 'accs', accuracies, 'epoch', epoch, 'cycle' ,cycle)



    # if False and epoch % 5 == 4:
    #     acc = test(models, dataloaders, 'test')
    #     if best_acc < acc:
    #         best_acc = acc
    #         torch.save({
    #             'epoch': epoch + 1,
    #             'state_dict_backbone': models['backbone'].state_dict(),
    #             'state_dict_module': models['module'].state_dict()
    #         },
    #         '%s/active_resnet18_cifar10.pth' % (checkpoint_dir))
    #     print('Val Acc: {:.3f} \t Best Acc: {:.3f}'.format(acc, best_acc))

    torch.save(models['backbone'].state_dict(), "model_" + str(cycle) + ".pt")
    print('>> Finished.')

In [19]:
def test(models, dataloaders, mode='val'):
    assert mode == 'val' or mode == 'test'
    models['backbone'].eval()
    # models['module'].eval()

    total = 0
    correct = 0
    with torch.no_grad():
        for (inputs, labels) in dataloaders[mode]:
            inputs = inputs.cuda()
            labels = labels.cuda()

            scores, _, _ = models['backbone'](inputs)
            _, preds = torch.max(scores.data, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    
    return 100 * correct / total

#
#
def get_uncertainty(models, unlabeled_loader):
    models['backbone'].eval()
    models['module'].eval()
    uncertainty = torch.tensor([]).cuda()

    with torch.no_grad():
        for (inputs, labels) in unlabeled_loader:
            inputs = inputs.cuda()
            # labels = labels.cuda()

            scores, features = models['backbone'](inputs)
            pred_loss = models['module'](features) # pred_loss = criterion(scores, labels) # ground truth loss
            pred_loss = pred_loss.view(pred_loss.size(0))

            uncertainty = torch.cat((uncertainty, pred_loss), 0)
    
    return uncertainty.cpu()


In [23]:
##
# Main
# if __name__ == '__main__':
# args = arguments.get_args()
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"
# main(args)
# vis = visdom.Visdom(server='http://localhost', port=9000)
vis = None
plot_data = {'X': [], 'Y': [], 'legend': ['Backbone Loss', 'Module Loss', 'Total Loss']}

for trial in range(TRIALS):
    # Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.

    all_indices = set(np.arange(NUM_TRAIN))
    val_indices = random.sample(all_indices, 0)
    all_indices = np.setdiff1d(list(all_indices), val_indices)

    initial_indices = random.sample(list(all_indices), ADDENDUM)
    f = open("./init_indice.pkl", 'wb')
    pickle.dump(initial_indices, f)
    # indices = all_indices
    # random.shuffle(indices)
    # labeled_set = indices[:ADDENDUM]

    # unlabeled_set = indices[ADDENDUM:]

    current_indices = list(initial_indices)

    train_loader = DataLoader(train_data, batch_size=BATCH, 
                                  sampler=SubsetRandomSampler(initial_indices), 
                                  pin_memory=True)
    test_loader  = DataLoader(test_data, batch_size=BATCH)
    val_loader = DataLoader(train_data, batch_size=BATCH, 
                              sampler=SubsetRandomSampler(val_indices), 
                              pin_memory=True)
    dataloaders  = {'train': train_loader, 'test': test_loader, 'val': val_loader}

    # Model
    # resnet18    = resnet.ResNet18(num_classes=10).cuda()
    # # loss_module = lossnet.LossNet().cuda()
    # models      = {'backbone': resnet18}
    # torch.backends.cudnn.benchmark = False

    # Active learning cycles

#         for cycle in range(CYCLES):
    # Loss, criterion and scheduler (re)initialization

    # Randomly sample 10000 unlabeled data points
    # random.shuffle(unlabeled_set)
    unlabeled_indices = np.setdiff1d(list(all_indices), current_indices)
    # Create unlabeled dataloader for the unlabeled subset
    unlabeled_loader = DataLoader(train_data, batch_size=BATCH, 
                                  sampler=SubsetSequentialSampler(unlabeled_indices), # more convenient if we maintain the order of subset
                                  pin_memory=True)

    resnet18    = resnet.ResNet18(num_classes=101).cuda()
    # loss_module = lossnet.LossNet().cuda()
    models      = {'backbone': resnet18}
    criterion      = nn.CrossEntropyLoss()
    optim_backbone = optim.SGD(models['backbone'].parameters(), lr=LR, 
                            momentum=MOMENTUM, weight_decay=WDECAY)
    # optim_module   = optim.SGD(models['module'].parameters(), lr=LR, 
    #                         momentum=MOMENTUM, weight_decay=WDECAY)
    sched_backbone = lr_scheduler.MultiStepLR(optim_backbone, milestones=MILESTONES)
    # sched_module   = lr_scheduler.MultiStepLR(optim_module, milestones=MILESTONES)

    optimizers = {'backbone': optim_backbone}
    schedulers = {'backbone': sched_backbone}

    # Training and test
    train(models, criterion, optimizers, schedulers, dataloaders, EPOCH, EPOCHL, vis, plot_data, 0, accuracies)
    acc = test(models, dataloaders, mode='test')
    accuracies.append(acc)
    print('Trial {}/{} || Cycle {}/{} || Label set size {}: Test acc {}'.format(trial+1, TRIALS, cycle+1, CYCLES, len(current_indices), acc))


    #semi


  0%|          | 0/63 [00:00<?, ?it/s]

>> Train a Model.


>> Finished.


RuntimeError: The size of tensor a (16) must match the size of tensor b (101) at non-singleton dimension 1

In [7]:
trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
# valloader = DataLoader(val_data, batch_size=16, shuffle=True)
testloader = DataLoader(test_data, batch_size=16, shuffle=False)

device = "cuda:0"
class ResNet34(nn.Module):
    def __init__(self, pretrained):
        super(ResNet34, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained=None)
        
        self.l0 = nn.Linear(512, len(lb.classes_))
        self.dropout = nn.Dropout2d(0.4)

    def forward(self, x):
        # get the batch size only, ignore (c, h, w)
        batch, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        x = self.dropout(x)
        l0 = self.l0(x)
        return l0

model = ResNet34(pretrained=False).to(device)
print(model)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# loss function
criterion = nn.CrossEntropyLoss()

# print(f"Training on {len(train_data)} examples, validating on {len(val_data)} examples")

#validation function
def validate(model, dataloader):
    print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            outputs = model(data)
            loss = criterion(outputs, torch.max(target, 1)[1])
            
            val_running_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            val_running_correct += (preds == torch.max(target, 1)[1]).sum().item()
        
        val_loss = val_running_loss/len(dataloader.dataset)
        val_accuracy = 100. * val_running_correct/len(dataloader.dataset)
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')
        
        return val_loss, val_accuracy

# training function
def fit(model, dataloader):
    print('Training')
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(data)
        # print(outputs)
        # print(torch.max(target, 1)[1])
        loss = criterion(outputs, torch.max(target, 1)[1])
        train_running_loss += loss.item()
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == torch.max(target, 1)[1]).sum().item()
        loss.backward()
        optimizer.step()
        
    train_loss = train_running_loss/len(dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(dataloader.dataset)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}")
    
    return train_loss, train_accuracy


train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []
start = time.time()
for epoch in range(1):
    print(f"Epoch {epoch+1} of {1}")
    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
    val_epoch_loss, val_epoch_accuracy = validate(model, valloader)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
end = time.time()

print((end-start)/60, 'minutes')

  0%|          | 0/433 [00:00<?, ?it/s]

ResNet34(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

434it [08:42,  1.20s/it]                         

Train Loss: 0.2185, Train Acc: 26.55


NameError: name 'valloader' is not defined

In [10]:
cifar10_train = Caltech101('./caltech', download=False, transform=train_transform)

In [11]:
test_loader  = DataLoader(cifar10_train, batch_size=BATCH)

In [12]:
trainloader = DataLoader(cifar10_train, batch_size=16, shuffle=True)
# valloader = DataLoader(val_data, batch_size=16, shuffle=True)
testloader = DataLoader(cifar10_train, batch_size=16, shuffle=False)

In [13]:
device = "cuda:0"
class ResNet34(nn.Module):
    def __init__(self, pretrained):
        super(ResNet34, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained='imagenet')
        else:
            self.model = pretrainedmodels.__dict__['resnet34'](pretrained=None)
        
        self.l0 = nn.Linear(512, 101)
        self.dropout = nn.Dropout2d(0.4)

    def forward(self, x):
        # get the batch size only, ignore (c, h, w)
        batch, _, _, _ = x.shape
        x = self.model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch, -1)
        x = self.dropout(x)
        l0 = self.l0(x)
        return l0

model = ResNet34(pretrained=False).to(device)
print(model)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# loss function
criterion = nn.CrossEntropyLoss()

# print(f"Training on {len(train_data)} examples, validating on {len(val_data)} examples")

#validation function
def validate(model, dataloader):
    print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            outputs = model(data)
            loss = criterion(outputs, torch.max(target, 1)[1])
            
            val_running_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            val_running_correct += (preds == torch.max(target, 1)[1]).sum().item()
        
        val_loss = val_running_loss/len(dataloader.dataset)
        val_accuracy = 100. * val_running_correct/len(dataloader.dataset)
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')
        
        return val_loss, val_accuracy

# training function
def fit(model, dataloader):
    print('Training')
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for data in tqdm(dataloader, leave=False, total=len(dataloader)):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(data)
        # print(outputs)
        # print(torch.max(target, 1)[1])
        loss = criterion(outputs, target)
        train_running_loss += loss.item()
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == torch.max(target, 1)[1]).sum().item()
        loss.backward()
        optimizer.step()
        
    train_loss = train_running_loss/len(dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(dataloader.dataset)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}")
    
    return train_loss, train_accuracy


train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []
start = time.time()
for epoch in range(1):
    print(f"Epoch {epoch+1} of {1}")
    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
    val_epoch_loss, val_epoch_accuracy = validate(model, valloader)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
end = time.time()

print((end-start)/60, 'minutes')

  0%|          | 0/543 [00:00<?, ?it/s]

ResNet34(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

TypeError: pic should be Tensor or ndarray. Got <class 'PIL.Image.Image'>.

In [2]:
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate
# from fastai.widgets import *
import re

In [3]:
PATH1 = Path('./caltech')
path = untar_data(URLs.CALTECH_101, dest=PATH1)
path.ls()

NameError: name 'Path' is not defined